In [ ]:
import ee
import numpy
import math
import sys
import time
sys.setrecursionlimit(30000)
ee.Authenticate()
ee.Initialize(project='yourself')

In [ ]:
################### Planting Date ########################
sowingYEAR = 2014
sowingYaD = sowingYEAR * 1000 + 250
harvestYaD = (sowingYEAR + 1) * 1000 + 190
start_year = str(sowingYaD)[:4]
sowway = 'projects/yourself/assets/new_sow' + start_year
sowdoy = ee.Image(sowway).subtract(250)


################### Bound information ########################
vector = ee.FeatureCollection('projects/yourself/assets/WinterWheat_Region')
bound = vector.geometry().bounds()
proj = {
    'scale': 1000,
    'crs': 'EPSG:4326'}


################### Variety data ########################
variety = ee.Image('projects/yourself/assets/variety').resample('bilinear').reproject(
    scale = proj['scale'],
    crs = proj['crs']).clip(bound).subtract(1)
varietylist = ee.List.sequence(0,18,1)
IElist = ee.List([0.997, 0.987, 0.86, 0.995, 0.994, 0.987, 0.994, 0.981, 0.943, 0.994, 0.994, 0.956, 0.987, 0.962, 0.74, 0.879, 0.74, 0.835, 0.943])
PVTlist = ee.List([57.1, 56.2, 58.1, 59.7, 50.5, 13.3, 14.3, 51.4, 28.6, 56.2, 53.3, 43.8, 38.2, 13.3, 12.4, 26.7, 3.8, 5.7, 2.9])
PSlist = ee.List([0.000929, 0.000913, 0.00089, 0.000173, 0.000772, 0.000882, 0.001, 0.000394, 0.000961, 0.000575, 0.001, 0.000992,
                  0.000976, 0.000882, 0.00074, 0.000323, 0.000882, 0.000858, 0.000205])
TSlist = ee.List([1.04, 1.08, 1.25, 1.27, 1.29, 1.7, 1.69, 1.23, 1.31, 1.25, 1.46, 1.12, 1.21, 1.69, 1.02, 1.01, 1.04, 1.27, 1.65])
FDFlist = ee.List([0.935, 0.987, 0.996, 0.858, 0.916, 0.994, 0.948, 0.929, 0.813, 0.961, 0.994, 0.871, 0.987, 0.897, 0.839, 0.806, 0.8, 0.819, 0.8])
HIlist = ee.List([0.45, 0.43, 0.35, 0.39, 0.44, 0.45, 0.44, 0.45, 0.45, 0.43, 0.44, 0.38, 0.43, 0.49, 0.45, 0.35, 0.45, 0.38, 0.42])
SLAClist = ee.List([0.0025, 0.003, 0.0025, 0.003, 0.0028, 0.0035, 0.0032, 0.0032, 0.0022, 0.0022, 0.0025, 0.003, 0.0019, 0.0035, 0.0028, 0.0024, 0.0026, 0.0035, 0.0028])
def variety_function(varlist):
  def getvarvalue(value, varietyimg):
    listnum = ee.Number(value)
    Varietyimg = ee.Image(varietyimg).where(ee.Image(varietyimg).eq(listnum.add(1)), ee.Number(varlist.get(listnum)))
    return Varietyimg
  return ee.Image(varietylist.iterate(getvarvalue, variety))

IE = variety_function(IElist)
PVT = variety_function(PVTlist)
PS = variety_function(PSlist)
TS = variety_function(TSlist)
FDF = variety_function(FDFlist)
HI = variety_function(HIlist)
SLAC = variety_function(SLAClist)

In [ ]:
# Generate an empty image with latitude and longitude information
LATimage = ee.Image.pixelLonLat().clip(bound).select('latitude').rename('LAT').reproject(
        scale = proj['scale'],
        crs = proj['crs']).clip(bound)


################### Calculation of the reproductive period (PDT) / WheatDevelopmentModel ########################
########## Getting information on Julian calendar days #########
def GetDOY(sowingYaD, harvestYaD):
  start_year = int(str(sowingYaD)[:4])
  end_year = int(str(harvestYaD)[:4])
  if start_year % 4 == 0 and (start_year % 100 != 0 or start_year % 400 == 0):
    year_days = 366
  else:
    year_days = 365
  a = sowingYaD % 1000
  b = harvestYaD % 1000
  DOY1 = range(a, year_days + 1)
  DOY2 = range(1, b+1)
  DOY = ee.Array([doy for doy in DOY1] + [doy for doy in DOY2])
  return DOY


######################## Daily Deviation DEC ########################
def CalDEC(DOY):
  sin = ee.Array(math.sin(23.45 * math.pi / 180))
  cos = DOY.add(10).multiply((2 * math.pi)).divide(365).cos()
  SC = sin.multiply(cos)
  tempDEC = SC.multiply(-1).asin()
  return ee.List(tempDEC)


######################## Intermediate variable SSIN、CCOS、SSCC ########################
def CalSSIN(LATimage, DEC):
  LAT = LATimage.select('LAT')
  DEC = DEC.getInfo()
  SSIN_list = []
  i = 0
  while i < num:
    SSIN = LAT.expression(
        'sin(Lat * (pi / 180)) * sin(DEC)',{
            'Lat': LAT,
            'pi': math.pi,
            'DEC': DEC[i]
            }).rename('ssin')
    SSIN_list.append(SSIN)
    i += 1
  return ee.ImageCollection(SSIN_list)


def CalCCOS(LATimage, DEC):
  LAT = LATimage.select('LAT')
  DEC = DEC.getInfo()
  CCOS_list = []
  i = 0
  while i < num:
    CCOS = LAT.expression(
        'cos(Lat * (pi / 180)) * cos(DEC)',{
            'Lat': LAT,
            'pi': math.pi,
            'DEC': DEC[i]
            }).rename('ccos')
    CCOS_list.append(CCOS)
    i += 1
  return ee.ImageCollection(CCOS_list)


def CalSSCC(SSIN, CCOS):
  merged_collection = SSIN.map(lambda image: image.addBands(CCOS.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calcsscc(image):
    ssin = image.select('ssin')
    ccos = image.select('ccos')
    sscc = ssin.divide(ccos)
    return sscc.rename('sscc')
  return merged_collection.map(calcsscc)


#################### Length of day DL ########################
def CalDL(SSCC):
  def caldl(sscc):
    DL = sscc.expression(
        '12 * (1 + 2 * asin(SSCC) / pi) + 40/60',{
            'SSCC': sscc,
            'pi': math.pi
            }).rename('dl')
    return DL
  return SSCC.map(caldl)


################### Calculation of relative photoperiodic effect RPE ########################
def CalRPE(PS, DL):
  def calrpe(dl):
    dayRPE = dl.expression(
        '1.0 - PS * pow(DL - 20, 2)',{
            'PS': PS,
            'DL': dl
            })
    return dayRPE
  return DL.map(calrpe)


############### Daily solar effective altitude DSINBE ########################
def CalDSINBE(DL, SSIN, CCOS, SSCC):
  merged_collection = SSIN.map(lambda image: image.addBands(CCOS.filterMetadata('system:index', 'equals', image.get('system:index')).first()) \
                            .addBands(SSCC.filterMetadata('system:index', 'equals', image.get('system:index')).first()) \
                            .addBands(DL.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def caldsinbe(image):
    ssin = image.select('ssin')
    ccos = image.select('ccos')
    sscc = image.select('sscc')
    dl = image.select('dl')
    DSINBE_1 = ssin.expression(
        'SSIN + 0.4 * (pow(SSIN, 2) + pow(CCOS, 2) * 0.5)',{
            'SSIN': ssin,
            'CCOS': ccos
            })
    DSINBE_2 = ssin.expression(
        'CCOS * (2 + 3 * 0.4 * SSIN) * sqrt(1 - pow(SSCC, 2)) / pi',{
            'CCOS': ccos,
            'SSIN': ssin,
            'SSCC': sscc,
            'pi': math.pi
            })
    DSINBE = dl.expression(
        '3600 * (DL * DSINBE_1 + 12 * DSINBE_2)',{
            'DL': dl,
            'DSINBE_1': DSINBE_1,
            'DSINBE_2': DSINBE_2
            })
    return DSINBE.rename('dsinbe')
  return merged_collection.map(caldsinbe)


######################## Solar time Th_j ########################
def CalTh_j(DL):
  DIS3 = [0.112702, 0.5, 0.887298]
  def calth_j(dl):
    Th_j_bandlist = []
    j = 0
    while j < 3:
      Th_j_j = dl.expression(
          '12 + 0.5 * DL * DIS3',{
              'DL': dl,
              'DIS3': DIS3[j]
              }).rename('th_j_' + str(j))
      Th_j_bandlist.append(Th_j_j)
      j += 1
    Th_j_image = ee.Image.cat(Th_j_bandlist)
    return Th_j_image
  return DL.map(calth_j)


######################## Sine of solar altitude angle at the jth moment of day i SINB_ij ########################
def CalSINB_ij(SSIN, CCOS, Th_j):
  merged_collection = SSIN.map(lambda image: image.addBands(CCOS.filterMetadata('system:index', 'equals', image.get('system:index')).first()) \
                            .addBands(Th_j.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calsinb_ij(image):
    ssin = image.select('ssin')
    ccos = image.select('ccos')
    SINB_ij_bandlist = []
    j = 0
    while j < 3:
      th_j = image.select('th_j_' + str(j))
      SINB_ij_j = ssin.expression(
        'SSIN + CCOS * cos(2 * pi * (Th_j + 12) / 24)',{
            'SSIN': ssin,
            'CCOS': ccos,
            'pi': math.pi,
            'Th_j': th_j
            }).rename('sinb_ij_' + str(j))
      SINB_ij_bandlist.append(SINB_ij_j)
      j += 1
    SINB_ij_image = ee.Image.cat(SINB_ij_bandlist)
    return SINB_ij_image
  return merged_collection.map(calsinb_ij)


######################## The jth moment canopy extinction coefficient K_j ########################
def CalK_j(SINB_ij):
  Con = ee.Image(0.42).clip(bound)
  def calk_j(image):
    K_j_bandlist = []
    j = 0
    while j < 3:
      sinb_ij = image.select('sinb_ij_' + str(j))
      K_j_j = Con.divide(sinb_ij).rename('k_j_' + str(j))
      K_j_bandlist.append(K_j_j)
      j += 1
    K_j_image = ee.Image.cat(K_j_bandlist)
    return K_j_image
  return SINB_ij.map(calk_j)


######################## Reflectance of light from the canopy at the jth moment p_j ########################
def Calp_j(SINB_ij):
  o = 0.2
  def calp_j(image):
    p_j_bandlist = []
    j = 0
    while j < 3:
      sinb_ij = image.select('sinb_ij_' + str(j))
      p_j_j = sinb_ij.expression(
          '((1 - sqrt(1 - o)) / (1 + sqrt(1 - o))) * (2.0 / (1 + 1.6 * SINB_ij))',{
              'o': o,
              'SINB_ij': sinb_ij
              }).rename('p_j_' + str(j))
      p_j_bandlist.append(p_j_j)
      j += 1
    p_j_image = ee.Image.cat(p_j_bandlist)
    return p_j_image
  return SINB_ij.map(calp_j)


################### SolarRadiation ERA ########################
def CalERAQ(sowingYaD):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  Q_list = []
  i = 0
  def resample_image(image):
    return image.resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs'])
  while i < num:
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-8, 'hour')
    end = end_date.advance(-8, 'hour')
    Radiation_colllection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).select('surface_solar_radiation_downwards_hourly')
    Radiation_colllection = Radiation_colllection.map(resample_image)
    dailyQ = Radiation_colllection.reduce(ee.Reducer.sum()).multiply(0.522).clip(bound).rename('q')
    Q_list.append(dailyQ)
    start_date = end_date
    i += 1
  return ee.ImageCollection(Q_list)


######################## Photosynthetic Active Radiation PAR_i ########################
def CalPAR_i(Q, SINB_ij, DSINBE):
  merged_collection = DSINBE.map(lambda image: image.addBands(SINB_ij.filterMetadata('system:index', 'equals', image.get('system:index')).first())) \
                 .map(lambda image: image.addBands(Q.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calpar_i(image):
    PAR_i_bandlist = []
    dsinbe = image.select('dsinbe')
    q = image.select('q')
    j = 0
    while j < 3:
      sinb_ij = image.select('sinb_ij_' + str(j))
      PAR_i_j = sinb_ij.expression(
          '0.5 * Q * SINB_ij * (1.0 + 0.4 * SINB_ij) / DSINBE',{
              'Q': q,
              'SINB_ij': sinb_ij,
              'DSINBE': dsinbe
          }).rename('par_i_' + str(j))
      PAR_i_bandlist.append(PAR_i_j)
      j += 1
    PAR_i_image = ee.Image.cat(PAR_i_bandlist)
    return PAR_i_image
  return merged_collection.map(calpar_i)


################### Calculate the average temperature T24H ########################
def CalT24H(sowingYaD):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  T24H_list = []
  i = 0
  def kelvin_to_celsius(image):
    return image.subtract(273.15).resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs'])
  while i < num:
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-8, 'hour')
    end = end_date.advance(-8, 'hour')
    Temperature_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).select(['temperature_2m'])
    Temperature_collection = Temperature_collection.map(kelvin_to_celsius)
    dailyT = Temperature_collection.reduce(ee.Reducer.mean()).clip(bound)
    T24H_list.append(dailyT)
    start_date = end_date
    i += 1
  return ee.ImageCollection(T24H_list)


################### Temperature per time period Ti ########################
def CalTi(sowingYaD):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  Ti_list = []
  def kelvin_to_celsius(image):
    return image.subtract(273.15).resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs']).clip(bound)
  i = 0
  while i < num:
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-8, 'hour')
    end = end_date.advance(-8, 'hour')
    Temperature_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY').filterDate(start, end).select(['temperature_2m'])
    Temperature_collection = Temperature_collection.map(kelvin_to_celsius)
    Ti_image = Temperature_collection.toBands()
    Ti_list.append(Ti_image)
    start_date = end_date
    i += 1
  return ee.ImageCollection(Ti_list)


################### Daily Tmax、Tmin ########################
def GetdayTmax(Ti):
  def caltmax(image):
    tmax = image.reduce(ee.Reducer.max())
    return tmax.rename('tmax_d')
  return Ti.map(caltmax)

def GetdayTmin(Ti):
  def caltmin(image):
    tmin = image.reduce(ee.Reducer.min())
    return tmin.rename('tmin_d')
  return Ti.map(caltmin)


####################### Calculate the average daily temperature ########################
def CaldTmean(Tmax, Tmin):
  merged_collection = Tmax.map(lambda image: image.addBands(Tmin.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def caldtmean(image):
    tmax = image.select('tmax_d')
    tmin = image.select('tmin_d')
    dT = tmax.multiply(0.6).add(tmin.multiply(0.4))
    return dT
  return merged_collection.map(caldtmean)


################### Calculation of daily vernalization effect DVE ########################
def CalDVE(PVT, Ti):
  vef = ee.Image(1).divide(ee.Image(2).subtract(ee.Image(0.0167).multiply(PVT))).clip(bound)
  Tou = ee.Image(10).subtract(PVT.divide(20)).clip(bound)
  Tmv = ee.Image(18).subtract(PVT.divide(8)).clip(bound)
  Tbv = -1
  Tol = 1
  def caldve(dayT):
    dayDVE = ee.Image(0).clip(bound)
    j = 0
    while j < 9:
      DVEI = dayT.expression(
          '((Tbv <= dayT) && (dayT < Tol)) ? pow(sin(((dayT - Tbv) * pi / ((Tol - Tbv) * 2))), 0.5) ' +
          ': ((Tol <= dayT) && (dayT < Tou)) ? 1 ' +
          ': ((Tou <= dayT) && (dayT <= Tmv)) ? cos((dayT - Tou) * pi / ((Tmv - Tou) * 2)) > 0 ? \
          pow(cos((dayT - Tou) * pi / ((Tmv - Tou) * 2)), vef) : pow(0.122887278627, vef) ' +
          ': 0 ' ,{
              'Tbv': Tbv,
              'dayT': dayT.select(j),
              'Tol': Tol,
              'pi': math.pi,
              'Tou': Tou,
              'Tmv': Tmv,
              'vef': vef
              })
      dayDVE = dayDVE.add(DVEI)
      j += 1
    tem_DVE = dayDVE.divide(9)
    return tem_DVE
  return Ti.map(caldve)


################### Daily devernalization effect DDEVE ########################
def CalDDEVE(Ti):
  def calddeve(ti):
    dayDDEVE = ee.Image(0).clip(bound)
    j = 0
    while j < 9:
      DEVEI = ti.expression(
          'dayT > 27 ? (dayT - 27) * 0.5 : 0',{
          'dayT': ti.select(j)
      })
      dayDDEVE = dayDDEVE.add(DEVEI)
      j += 1
    tem_DDEVE = dayDDEVE.divide(9)
    return tem_DDEVE
  return Ti.map(calddeve)


################### Vernalization effect VD ########################
def CalVD(sowdoy, PVT, DDEVE, DVE):
  VD_list = []
  tempPVT = PVT.multiply(0.3)
  sumVD = ee.Image(0).clip(bound)
  i = 0
  while i < num:
    dve = DVE.filterMetadata('system:index', 'equals', str(i)).first()
    ddeve = DDEVE.filterMetadata('system:index', 'equals', str(i)).first()
    sumVD = sumVD.expression(
        'i <= sowdoy ? 0 ' +
        ': ((0 <= sumVD) && (sumVD <= tempPVT)) ? (DVE - DDEVE) > 0 ? (sumVD + DVE - DDEVE) : sumVD' +
        ': ((tempPVT < sumVD) && (sumVD <= PVT)) ? (sumVD + DVE) : sumVD',{
            'i': i,
            'sowdoy': sowdoy,
            'sumVD': sumVD,
            'tempPVT': tempPVT,
            'DDEVE': ddeve,
            'PVT': PVT,
            'DVE': dve
            }).clip(bound)
    VD_list.append(sumVD)
    i += 1
  return ee.ImageCollection(VD_list)


################# Vernalization process VP ########################
def CalVP(VD, PVT):
  def calvp(vd):
    dayVP = vd.expression(
        '(VD / PVT) <= 1 ? VD / PVT : 1',{
            'VD': vd,
            'PVT': PVT
        })
    return dayVP.clip(bound)
  return VD.map(calvp)


#################### Factors affecting the maximum rate of photosynthesis by mean daily temperature FTMP #######################
def CalFTMP(T24H):
  Tb = 0
  Tol = 15
  Tou = 25
  Tmax = 45
  def calftmp(t24h):
    tempFTMP = t24h.expression(
        '(Tb <= T24H) && (T24H < Tol) ? sin(((T24H - Tb) / (Tol - Tb)) * pi / 2) ' +
        ': (Tol <= T24H) && (T24H < Tou) ? 1 ' +
        ': (Tou <= T24H) && (T24H < Tmax) ? cos(((T24H - Tou) / (Tmax - Tou)) * pi / 2) ' +
        ': 0.001 ',{
            'Tb': Tb,
            'T24H': t24h,
            'Tol': Tol,
            'pi': math.pi,
            'Tou': Tou,
            'Tmax': Tmax
        })
    return tempFTMP
  return T24H.map(calftmp)


################### GDD, PDT, FA, PIS, PILVG, PISP, SLA, AMAX, LwpCr ########################
def CalPDT(PlantingDepth, RPE, VP, PVT, VD, FDF, Ti, TS, IE, sowdoy, FTMP, FCO2):
  DayGDD_list = []
  PDT_list = []
  FA_list = []
  PIS_list = []
  PILV_list = []
  PISP_list = []
  SLA_list = []
  AMAX_list = []
  LwpCr_list = []
  PDTTS = 16.1
  PDTHD = 26.8
  AMX = ee.Image(40).clip(bound)
  EMGDD = 60 + 30 * PlantingDepth
  DPE = ee.Image(0).clip(bound)
  tempPDT = ee.Image(0).clip(bound)
  TempSumDTT = ee.Image(0).clip(bound)
  daySLA = ee.Image(0.002171).clip(bound)
  i = 0
  while i < num:
    Tb = ee.Image(0).clip(bound).expression(
        'tempPDT < 9.5 ? 0 : tempPDT < 26.8 ? 3.3 : tempPDT < 39 ? 8 : 8',{
            'tempPDT': tempPDT})
    To = ee.Image(0).clip(bound).expression(
        'tempPDT < 9.5 ? 20 : tempPDT < 26.8 ? 22 : tempPDT < 39 ? 25 : 25',{
            'tempPDT': tempPDT})
    Tm = ee.Image(0).clip(bound).expression(
        'tempPDT < 9.5 ? 32 : tempPDT < 26.8 ? 32 : tempPDT < 39 ? 35 : 35',{
            'tempPDT': tempPDT})
    dayGDD = ee.Image(0).clip(bound)
    dayT = Ti.filterMetadata('system:index', 'equals', str(i)).first()
    SumTE_i = ee.Image(0).clip(bound)
    vd = VD.filterMetadata('system:index', 'equals', str(i)).first()
    rpe = RPE.filterMetadata('system:index', 'equals', str(i)).first()
    vp = VP.filterMetadata('system:index', 'equals', str(i)).first()
    ftmp = FTMP.filterMetadata('system:index', 'equals', str(i)).first()
    for j in range(9):
      RTEI = Tb.expression(
          '((Tb <= dayT) && (dayT <= To)) ? pow(sin((dayT - Tb) * pi / (2 * (To - Tb))), TS)' +
          ': ((To <= dayT) && (dayT <= Tm)) ? tempPDT < 26.8 ? pow(cos(pow((dayT - To) * pi / ((Tm - To) * 2), (Tm - To) / (To - Tb))), TS)' +
          ': pow(sin((Tm - dayT) * pi / ((Tm - To) * 2)), ((Tm - To) / (To - Tb)) * TS)' +
          ': 0 ',{
              'Tb': Tb,
              'dayT': dayT.select(j),
              'To': To,
              'pi': math.pi,
              'TS': TS,
              'Tm': Tm,
              'tempPDT': tempPDT
          })
      SumTE_i = SumTE_i.add(RTEI)
      TempSumDTT = TempSumDTT.expression(
            'i >= sowdoy ? dayT >= Tb ? TempSumDTT + (dayT - Tb) : TempSumDTT : 0',{
                'i': i,
                'sowdoy': sowdoy,
                'dayT': dayT.select(j),
                'Tb': Tb,
                'TempSumDTT': TempSumDTT
            })
      dayGDD = dayGDD.expression(
        'dayT >= Tb ? dayGDD + (dayT - Tb) : dayGDD ',{
            'dayT': dayT.select(j),
            'Tb': Tb,
            'dayGDD': dayGDD
        })
    DTE = SumTE_i.divide(9).clip(bound)
    GDD = TempSumDTT.divide(9).clip(bound)

    ######################## Daily GDD ########################
    DayGDD = dayGDD.divide(9).clip(bound)
    DayGDD_list.append(DayGDD)
    DTS = vd.expression(
        'VD < PVT ? RPE * VP : ((VD >= PVT) && (tempPDT <= PDTTS)) ? RPE : ((PDTTS < tempPDT) && (tempPDT < PDTHD)) ? ' +
        ' RPE + (1 - RPE) * (tempPDT - PDTTS) / (PDTHD - PDTTS)' +
        ': 1 ',{
            'VD': vd,
            'PVT': PVT,
            'RPE': rpe,
            'VP': vp,
            'tempPDT': tempPDT,
            'PDTTS': PDTTS,
            'PDTHD': PDTHD
        })
    DPE = GDD.expression(
        'GDD >= EMGDD ? DTE * DTS * IE * 0.6 : 0',{
            'GDD': GDD,
            'EMGDD': EMGDD,
            'DTE': DTE,
            'DTS': DTS,
            'IE': IE
        })
    tempPDT = tempPDT.expression(
        'tempPDT < 26.8 ? tempPDT + DPE : tempPDT < 39 ? tempPDT + DTE : tempPDT < 56 ? tempPDT + DTE * FDF : 56',{
            'tempPDT': tempPDT,
            'DPE': DPE,
            'DTE': DTE,
            'FDF': FDF
        }).clip(bound)
    PDT_list.append(tempPDT)

    ################### Factors influencing the maximum rate of photosynthesis by physiological age FA ########################
    tempFA = tempPDT.expression(
        'PDT >= 31 ? 1 - 0.02 * (PDT - 31) : 1',{
            'PDT': tempPDT,
        }).clip(bound)
    FA_list.append(tempFA)

    ######################## Ground allocation index PIS ########################
    tempPIS = tempPDT.expression(
    'PDT > 0.2 ? 0.8819 / (1 + 1.215 * exp(-0.0476 * PDT)) : 0',{
        'e': math.e,
        'PDT': tempPDT
    }).clip(bound)
    PIS_list.append(tempPIS)

    ######################## Green leaf allocation index PILV ########################
    tempPILV = tempPDT.expression(
    'PDT <= 56 ? 0.809 / (1 + 51.4 * exp(-0.112 * (56 - PDT))) : 0',{
        'PDT': tempPDT,
        'e': math.e
    }).clip(bound)
    PILV_list.append(tempPILV)

    ######################## Spike allocation index PISP ########################
    tempPISP = tempPDT.expression(
        'PDT >= 7.4 ? (0.11 + HI) * pow(sin((PDT - 7.4) * pi / (46.6 * 2)), (4 + HI)) : 0',{
            'PDT': tempPDT,
            'HI': HI,
            'pi': math.pi
    }).clip(bound)
    PISP_list.append(tempPISP)

    ######################## Specific leaf area SLA ########################
    DAYSLA = daySLA
    daySLA = tempPDT.expression(
        'PDT < 56 ? SLAc * exp(-0.73 / (56 - PDT)) : SLAc * exp(-0.73)',{
            'PDT': tempPDT,
            'SLAc': SLAC
        })
    SLA = daySLA.expression(
      '(PDT > 55) && (SLA_Y < SLA) ? SLA_Y : SLA',{
          'PDT': tempPDT,
          'SLA_Y': DAYSLA,
          'SLA': daySLA
      }).clip(bound)
    SLA_list.append(SLA)

    ######################## Maximum photosynthesis rate of a single leaf AMAX ########################
    tempAMAX = AMX.multiply(ftmp).multiply(tempFA).multiply(FCO2).clip(bound).rename('amax')
    AMAX_list.append(tempAMAX)

    ######################## Early morning critical leaf water potential LwpCr ########################
    tempLwpCr = ee.Image(0).expression(
        'PDT < 16.1 ? -10 ' +
         ': (16.1 <= PDT) && (PDT < 21.4) ? -8 ' +
         ': (21.4 <= PDT) && (PDT < 31) ? -9 ' +
         ': -12',{
             'PDT':tempPDT})
    LwpCr_list.append(tempLwpCr)
    i += 1
  return ee.ImageCollection(DayGDD_list), ee.ImageCollection(PDT_list), ee.ImageCollection(FA_list), ee.ImageCollection(PIS_list), ee.ImageCollection(PILV_list), \
      ee.ImageCollection(PISP_list), ee.ImageCollection(SLA_list), ee.ImageCollection(AMAX_list), ee.ImageCollection(LwpCr_list)


################################################ Moisture Module ################################################


####################### Soil bulk ########################
def Getopendata(image):
  b_0 = image.select(['b0'])
  b_10 = image.select(['b10'])
  b_30 = image.select(['b30'])
  b_60 = image.select(['b60'])
  b_100 = image.select(['b100'])
  b_200 = image.select(['b200'])
  b_30 = (b_0.add(b_60)).divide(2)
  b_20 = (b_30.add(b_10)).divide(2).rename('b20')
  b_40 = (b_20.add(b_60)).divide(2).rename('b40')
  b_80 = (b_60.add(b_100)).divide(2).rename('b80')
  b_150 = (b_100.add(b_200)).divide(2)
  b_90 = (b_80.add(b_100)).divide(2)
  b_120 = (b_90.add(b_150)).divide(2).rename('b120')
  b_160 = (b_120.add(b_200)).divide(2).rename('b160')
  b_140 = (b_120.add(b_160)).divide(2).rename('b140')
  b_180 = (b_160.add(b_200)).divide(2).rename('b180')
  b_0 = b_0.addBands(b_20).addBands(b_40).addBands(b_60).addBands(b_80).addBands(b_100) \
          .addBands(b_120).addBands(b_140).addBands(b_160).addBands(b_180).divide(100).resample('bilinear').reproject(
          scale = proj['scale'],
          crs = proj['crs'])
  return b_0.clip(bound)


################### Saturated water content of the soil layer ########################
def Calsat(dBD):
  sat_list = []
  for j in range(10):
    dbd = dBD.select(j)
    tempsat = dbd.expression(
        'abs(1 - dBD / 2.65)',{
            'dBD': dbd
        })
    sat_list.append(tempsat)
  return ee.Image.cat(sat_list)


################### Saturated hydraulic conductivity of soil ########################
def CalKSat(sat, dul):
  KSat_list = []
  for i in range(10):
    SAT = sat.select(i)
    DUL = dul.select(i)
    tempKSat = dul.expression(
        '75 * pow((sat - dul) / dul, 2)',{
            'sat': SAT,
            'dul': DUL
        })
    KSat_list.append(tempKSat)
  return ee.Image.cat(KSat_list)


################### Intermediate variables in crop transpiration soil evapotranspiration ###################
def CalKt(Tmax):
  def calkt(image):
    tempKt = image.expression(
        'Tmax < 5 ? 0.01 * exp(0.18 * (Tmax + 20)) ' +
        ': Tmax <= 24 ? 1.1 ' +
        ': 0.05 * (Tmax - 24) + 1.1',{
            'Tmax': image
        }
    )
    return tempKt
  return Tmax.map(calkt)


################### Apoptotic coefficient , field water holding capacity ########################
def calll(sand, clay, org, band, newname):
  Sand = sand.multiply(0.01).select(band)
  Clay = clay.multiply(0.01).select(band)
  ORG = org.multiply(0.005).select(band)
  theta_1500ti = ee.Image(0).expression("-0.024 * S + 0.487 * C + 0.006 * OM + 0.005 * (S * OM)\
  - 0.013 * (C * OM) + 0.068 * (S * C) + 0.031",{
      "S": Sand,
      "C": Clay,
      "OM": ORG})
  pwp = theta_1500ti.expression("T1500ti + ( 0.14 * T1500ti - 0.002)", {"T1500ti": theta_1500ti}).rename(newname)
  return pwp.resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs']).clip(bound)

def caldul(sand, clay, org, band, newname):
  Sand = sand.multiply(0.01).select(band)
  Clay = clay.multiply(0.01).select(band)
  ORG = org.multiply(0.005).select(band)
  theta_33ti = ee.Image(0).expression("-0.251 * S + 0.195 * C + 0.011 * OM + 0.006 * (S * OM) - 0.027 * (C * OM)+\
  0.452 * (S * C) + 0.299",{
      "S": Sand,
      "C": Clay,
      "OM": ORG})
  fc = theta_33ti.expression("T33ti + (1.283 * T33ti * T33ti - 0.374 * T33ti - 0.015)",{"T33ti": theta_33ti}).rename(newname)
  return fc.resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs']).clip(bound)


def Getlldata(image):
  b_0 = image.select('b0')
  b_10 = image.select('b10')
  b_30 = image.select('b30')
  b_60 = image.select('b60')
  b_100 = image.select('b100')
  b_200 = image.select('b200')
  b_30 = (b_0.add(b_60)).divide(2)
  b_20 = (b_30.add(b_10)).divide(2).rename('b20')
  b_40 = (b_20.add(b_60)).divide(2).rename('b40')
  b_80 = (b_60.add(b_100)).divide(2).rename('b80')
  b_150 = (b_100.add(b_200)).divide(2)
  b_90 = (b_80.add(b_100)).divide(2)
  b_120 = (b_90.add(b_150)).divide(2).rename('b120')
  b_160 = (b_120.add(b_200)).divide(2).rename('b160')
  b_140 = (b_120.add(b_160)).divide(2).rename('b140')
  b_180 = (b_160.add(b_200)).divide(2).rename('b180')
  bx = b_0.addBands(b_20).addBands(b_40).addBands(b_60).addBands(b_80).addBands(b_100) \
          .addBands(b_120).addBands(b_140).addBands(b_160).addBands(b_180)
  return bx


######################## Root hair density ########################
def CaldRLV():
  dr = ee.Image(0)
  for l in range(9):
    dr = dr.addBands(ee.Image(0))
  return dr.clip(bound)

######################## Root preference facto ########################
def CalWR(LayersCount, Layer):  # (Number of soil layers, thickness of soil layers)
  WR = []
  TempLayersDepth = 0  # SOIL depth
  for j in range(LayersCount):
    z = TempLayersDepth + Layer / 2
    tempWR = numpy.exp(-4 * z / 200)
    TempLayersDepth = TempLayersDepth + Layer
    WR.append(tempWR)
  return WR

################### Calculation of soil water content ########################
def Getsw(sowingYaD, bandname, rate, newname, irrarea, addrate, PDT):
  start_year = int(str(sowingYaD)[:4])
  day_of_startyear = int(str(sowingYaD)[4:])
  start_date = ee.Date.fromYMD(start_year, 1, 1).advance(day_of_startyear-1, 'day').format('YYYY-MM-dd')
  start_date = ee.Date(start_date)
  sw_list = []
  i = 0
  def calmean(image):
    return image.divide(rate).resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs'])
  while i < num:
    pdt = PDT.filterMetadata('system:index', 'equals', str(i)).first()
    end_date = start_date.advance(1, 'day')
    start = start_date.advance(-10, 'hour')
    end = end_date.advance(-5, 'hour')
    sw_collection = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').filterDate(start, end).select(bandname)
    sw_collection = sw_collection.map(calmean)
    dailysw = sw_collection.reduce(ee.Reducer.mean()).clip(bound).rename(newname)
    dailysw = dailysw.expression(
    '(irrarea == 1) ? ' +
      '0 <= PDT && PDT < 16.2 ? ' +
        'dailysw < 0.8 * addrate ? ' +
          '0.8 * addrate : dailysw ' +
      ':16.2 <= PDT && PDT < 35 ? ' +
        'dailysw < 0.9 * addrate ? ' +
          '0.9 * addrate : dailysw ' +
      ': addrate '
    ': dailysw',{
        'PDT': pdt,
        'dailysw': dailysw,
        'irrarea': irrarea,
        'addrate': addrate
    })
    sw_list.append(dailysw)
    start_date = end_date
    i += 1
  return ee.ImageCollection(sw_list)

def Calsw(coll1, coll2, coll1name, coll2name, newname):
  merged_collection = coll1.map(lambda image: image.addBands(coll2.filterMetadata('system:index', 'equals', image.get('system:index')).first()))
  def calsw(image):
    sw1 = image.select(coll1name)
    sw2 = image.select(coll2name)
    sw_image_daily = sw1.add(sw2).divide(2)
    return sw_image_daily.rename(newname)
  return merged_collection.map(calsw)



################################################ Main Function ################################################

############## Management ##############

ABIOMASS = ee.Image(130).clip(bound)  # kg/ha  seedin
LayersCount = 10
Layer = 20
PlantingDepth = 2.5

############## Fertility PDT (Solar temperature module) ##############
DOY = GetDOY(sowingYaD, harvestYaD)
DEC = CalDEC(DOY)
num = len(DEC.getInfo())
SSIN = CalSSIN(LATimage, DEC)
CCOS = CalCCOS(LATimage, DEC)
SSCC = CalSSCC(SSIN, CCOS)
DL = CalDL(SSCC)
RPE = CalRPE(PS, DL)
DSINBE = CalDSINBE(DL, SSIN, CCOS, SSCC)
Th_j = CalTh_j(DL)
SINB_ij = CalSINB_ij(SSIN, CCOS, Th_j)
K_j = CalK_j(SINB_ij)
p_j = Calp_j(SINB_ij)
Q = CalERAQ(sowingYaD)
PAR_i = CalPAR_i(Q, SINB_ij, DSINBE)
T24H = CalT24H(sowingYaD)
Ti = CalTi(sowingYaD)
Tmax = GetdayTmax(Ti)
Tmin = GetdayTmin(Ti)
dTmean = CaldTmean(Tmax, Tmin)
DVE = CalDVE(PVT, Ti)
DDEVE = CalDDEVE(Ti)
VD = CalVD(sowdoy, PVT, DDEVE, DVE)
VP = CalVP(VD, PVT)
FTMP = CalFTMP(T24H)
FCO2 = ee.Image(1 + 0.8 * numpy.log(414.02/340)).clip(bound)
GDD, PDT, FA, PIS, PILVG, PISP, SLA, AMAX, LwpCr = CalPDT(PlantingDepth, RPE, VP, PVT, VD, FDF, Ti, TS, IE, sowdoy, FTMP, FCO2)

##############  Soil Moisture ##############

irr = ee.Image("USGS/GFSAD1000_V1").select('landcover').clip(bound)
irrarea = irr.updateMask(irr.gt(0)).updateMask(irr.lt(3)).where(irr.gt(1),1)
dul_image = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").clip(bound)
dBD_image = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").clip(bound)
sand = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").clip(bound)
clay = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").clip(bound)
org = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").clip(bound)
dul0 = caldul(sand, clay, org, ['b0'], 'b0')
dul10 = caldul(sand, clay, org, ['b10'], 'b10')
dul30 = caldul(sand, clay, org, ['b30'], 'b30')
dul60 = caldul(sand, clay, org, ['b60'], 'b60')
dul100 = caldul(sand, clay, org, ['b100'], 'b100')
dul200 = caldul(sand, clay, org, ['b200'], 'b200')
tempdul = dul0.addBands(dul10).addBands(dul30).addBands(dul60).addBands(dul100).addBands(dul200)

dul = Getlldata(tempdul)

ll0 = calll(sand, clay, org, ['b0'], 'b0')
ll10 = calll(sand, clay, org, ['b10'], 'b10')
ll30 = calll(sand, clay, org, ['b30'], 'b30')
ll60 = calll(sand, clay, org, ['b60'], 'b60')
ll100 = calll(sand, clay, org, ['b100'], 'b100')
ll200 = calll(sand, clay, org, ['b200'], 'b200')
ll0 = ll0.addBands(ll10).addBands(ll30).addBands(ll60).addBands(ll100).addBands(ll200)
ll = Getlldata(ll0)

dBD = Getopendata(dBD_image)
sat = Calsat(dBD)
KSat = CalKSat(sat, dul) ###This function is used to calculate rainfall infiltration
Kt = CalKt(Tmax)


# GLDAS
sw_0 = Getsw(sowingYaD, 'SoilMoi0_10cm_inst', 100, 'sw_0', irrarea, dul.select(0), PDT)
sw_2 = Getsw(sowingYaD,  'SoilMoi10_40cm_inst', 400, 'sw_40', irrarea, dul.select(3), PDT)
sw_5 = Getsw(sowingYaD,  'SoilMoi40_100cm_inst', 1000, 'sw_100', irrarea, dul.select(5), PDT)
sw_10 = Getsw(sowingYaD,  'SoilMoi100_200cm_inst', 2000, 'sw_200', irrarea, (dul_image.select(['b200']).divide(100)), PDT)
sw_1 = Calsw(sw_0, sw_2, 'sw_0', 'sw_40', 'sw_20')
sw_3 = Calsw(sw_1, sw_5, 'sw_20', 'sw_100', 'sw_60')
sw_4 = Calsw(sw_3, sw_5, 'sw_60', 'sw_100', 'sw_80')
sw_6 = Calsw(sw_2, sw_10, 'sw_40', 'sw_200', 'sw_120')
sw_7 = Calsw(sw_4, sw_10, 'sw_80', 'sw_200', 'sw_140')
sw_8 = Calsw(sw_6, sw_10, 'sw_120', 'sw_200', 'sw_160')
sw_9 = Calsw(sw_8, sw_10, 'sw_160', 'sw_200', 'sw_180')

############## Intermediate variable ##############
wDurDays = ee.Image(0).clip(bound)
dRLV = CaldRLV()
WR = CalWR(LayersCount, Layer)
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation').resample('bilinear').reproject(scale = proj['scale'], crs = proj['crs']).clip(bound)

In [ ]:
def WheatGrowModel(ABIOMASS, dRLV, Q, Kt, dTmean, LwpCr, sw_0, sw_1, sw_2, sw_3, sw_4, sw_5, sw_6, sw_7, sw_8, sw_9, ll, dul, sat, p_j, \
                   PAR_i, K_j, AMAX, DL, T24H, PIS, PILVG, WR, PISP, SLA, GDD, Layer, PDT, Num, YIELD, ATOPWT, AWLVG, WST, \
                   AROOTWT, WSP, yAWLVG, yWST, yAROOTWT, yWSP, LAI, RTDEP, flag_chumiao, dem, wDurDays, Laimax):

  ######################## Data initialization ########################
  Albedo = ee.Image(0.23).clip(bound)  # 裸土反射率
  WGUSS3 = [0.277778, 0.444444, 0.277778]
  DIS5 = [0.04691, 0.230753, 0.5, 0.769147, 0.95309]
  WGUSS5 = [0.118464, 0.239314, 0.284444, 0.239314, 0.118464]
  WDurDays = wDurDay
  ######################################################################################################################################################################
  for i in range(Num, Num + 20):
    m_Ta = ee.Image(0).clip(bound)
    pdt = PDT.filterMetadata('system:index', 'equals', str(i)).first()
    sla = SLA.filterMetadata('system:index', 'equals',str(i)).first()
    ######################## Data initialization after seedling emergence ########################
    AWLVG = flag_chumiao.expression(
        '(flag == 1) && (PDT > 0.2) ? ABIOMASS * 0.65 * 0.65 : AWLVG ',{
            'flag':flag_chumiao,
            'AWLVG': AWLVG,
            'PDT': pdt,
            'ABIOMASS':ABIOMASS
            }).clip(bound)
    WST = flag_chumiao.expression(
        '(flag == 1) && (PDT > 0.2) ? ABIOMASS * 0.65 * 0.35 : WST ',{
            'flag': flag_chumiao,
            'PDT': pdt,
            'WST': WST,
            'ABIOMASS':ABIOMASS
            }).clip(bound)
    ATOPWT = flag_chumiao.expression(
        '(flag == 1) && (PDT > 0.2) ? AWLVG + WST : ATOPWT ',{
            'flag': flag_chumiao,
            'PDT': pdt,
            'ATOPWT': ATOPWT,
            'WST': WST,
            'AWLVG':AWLVG,
            }).clip(bound)
    AROOTWT = flag_chumiao.expression(
        '(flag == 1) && (PDT > 0.2) ? ABIOMASS * 0.35 : AROOTWT ',{
            'flag': flag_chumiao,
            'PDT': pdt,
            'AROOTWT': AROOTWT,
            'ABIOMASS':ABIOMASS
            }).clip(bound)
    LAI = flag_chumiao.expression(
        '(flag == 1) && (PDT > 0.2) ? ABIOMASS * 0.7 / 2 * SLA : LAI ',{
            'flag': flag_chumiao,
            'PDT': pdt,
            'LAI': LAI,
            'ABIOMASS':ABIOMASS,
            'SLA':sla
            })
    flag_chumiao = flag_chumiao.expression(
        '(flag == 1) && (PDT > 0.2) ? -1 : flag',{
            'flag': flag_chumiao,
            'PDT': pdt })
    ################### Daily root length data dTotRLV ###################
    dTotRLV =  dRLV.select(9)
    for l in range(9):
      dTotRLV = dTotRLV.add(dRLV.select(l))
    dTotRLV = dTotRLV.where(dTotRLV.lte(0), 100000)
    ##################################### Crop transpiration Soil evapotranspiration ######################################
    Albedo = pdt.expression(
        'PDT > 16.2 ? 0.23 + pow(LAI, 2) / 160 : Albedo',{
            'PDT': pdt,
            'LAI': LAI,
            'Albedo':Albedo
            }).clip(bound)
    ################### Reference crop potential evapotranspiration m_ETpRe ###################
    q = Q.filterMetadata('system:index', 'equals', str(i)).first()
    kt = Kt.filterMetadata('system:index', 'equals', str(i)).first()
    dtmean = dTmean.filterMetadata('system:index', 'equals', str(i)).first()
    m_ETpRe = q.expression(
        'Kt * Q / 1000000 * (0.00488 - 0.00437 * Albedo) * (dTmean + 29)',{
            'Kt': kt,
            'Q': q,
            'Albedo': Albedo,
            'dTmean': dtmean
            })
    ################### Potential crop evapotranspiration m_ETp ###################
    m_ETp = LAI.expression(
        'LAI <= 1.5 ? m_ETpRe' +
        ': 1.5 < LAI < 5 ? ((1.22 - 1) * LAI + (5 - 1.5 * 1.22)) * m_ETpRe / 3.5 ' +
        ': 1.22 * m_ETpRe',{
            'LAI': LAI,
            'm_ETpRe': m_ETpRe
            }).clip(bound)
    ################### Potential soil evapotranspiration m_ESp && Potential crop transpiration m_Tp ###################
    m_ESp = m_ETp.expression(
        'm_ETp * exp(-0.5 * LAI)',{
            'm_ETp':m_ETp,
            'LAI': LAI
            })
    m_Tp = m_ETp.subtract(m_ESp)
    ###################################### Crop root water uptake m_Ta ######################################
    for l in range(10):
      sw_x = eval('sw_' + str(l)).filterMetadata('system:index', 'equals', str(i)).first()
      dRupWF = ll.expression(
          'sw <= ll ? 0 ' +
          ': ll < sw && sw < dul ? pow((sw - ll) / (dul - ll), 0.7) ' +
          ': 1 ',{
              'sw': sw_x,
              'll': ll.select(l),
              'dul': dul.select(l)
          }).clip(bound)
      drwu = m_Tp.multiply(dRupWF).multiply(dRLV.select(l)).divide(dTotRLV)

      m_Ta = m_Ta.add(drwu)
      if l == 0:
        dRWU = drwu
      else:
        dRWU = dRWU.addBands(drwu)
    ###################################### Calculate the moisture impact factor ######################################
    m_outputSWDF1 = ee.Image(1).clip(bound)
    m_outputSWDF2 = ee.Image(1).clip(bound)
    lwpcr = LwpCr.filterMetadata('system:index', 'equals', str(i)).first()
    tempSWF = ee.Image(0).clip(bound)

    WDurDays = dul.expression(
        'PDT > 0 ? sw > dul ? WDurDays + 1 : 0 : 0',{
            'PDT': pdt,
            'sw': sw_0.filterMetadata('system:index', 'equals', str(i)).first(),
            'dul': dul.select(0),
            'WDurDays': WDurDays
        }).clip(bound)
    WDurDays = WDurDays.where(WDurDays.gt(20), 20)

    ZSSWF = pdt.expression(
        '0 <= pdt && pdt < 16.1 ? -0.0000838 * pow(WDurDays,3) + 0.0032 * pow(WDurDays,2) - 0.0481 * WDurDays + 1.1255 : \
        16.1 <= pdt && pdt < 18.8 ? -0.0000519 * pow(WDurDays,3) + 0.003 * pow(WDurDays,2) - 0.0572 * WDurDays + 1.074 : \
        18.8 <= pdt && pdt < 21.4 ? -0.000143 * pow(WDurDays,3) + 0.0064 * pow(WDurDays,2) - 0.0957 * WDurDays + 1.2084 : \
        21.4 <= pdt && pdt < 39 ? 1 - 1/(1 + 29.9 * exp(-0.19 * WDurDays)) :\
        1  - 1/(1 + 105.61 * exp(-0.234 * WDurDays))',{
            'pdt': pdt,
            'WDurDays': WDurDays
        })

    for l in range(10):
      sw_x = eval('sw_' + str(l)).filterMetadata('system:index', 'equals', str(i)).first()
      SWcr = dul.expression(
          'PDT > 0 ? ll + (dul - ll) * 0.003 * exp(5.9165 * (30 + LwpCr) / 24) : 0',{
              'PDT': pdt,
              'll': ll.select(l),
              'dul': dul.select(l),
              'LwpCr': lwpcr
          }).clip(bound)
      dSWF = sw_x.expression(
          'PDT > 0 ? \
            sw > sat ? \
              ZSSWF * (sw - dul) / (sat - dul) : \
            (dul < sw) && (sw <= sat) ? \
              ZSSWF : \
            (SWcr < sw ) && (sw <= dul) ? \
              1.1 : \
            sw >= ll ? \
              (sw - ll) / (SWcr - ll) : \
              0 : \
            0',{
              'ZSSWF': ZSSWF,
              'PDT': pdt,
              'sw': sw_x,
              'sat': sat.select(l),
              'SWcr': SWcr,
              'dul': dul.select(l),
              'll': ll.select(l),
              'WDurDays': WDurDays
          }).clip(bound)
      tempSWF = tempSWF.expression(
          'm_Ta > 0 ? SWF + dSWF * dRWU / m_Ta : SWF',{
              'm_Ta': m_Ta,
              'SWF': tempSWF,
              'dSWF': dSWF,
              'dRWU': dRWU.select(l),
              'm_Ta': m_Ta
          }).clip(bound)
    SWF = tempSWF.where(tempSWF.lt(0.1), 0.1)
    m_outputSWDF1 = SWF
    m_outputSWDF2 = ee.Image(0.5).add(SWF.divide(2))
    ##################################### Soil Moisture Influence Factor per Layer ######################################
    M_DSWDF_9 = dul.expression(
        'dul != ll ? sw > ll ? (sw - ll) / (dul - ll) : 0 : 0 ',{
              'dul': dul.select(0),
              'll': ll.select(0),
              'sw': sw_0.filterMetadata('system:index', 'equals', str(i)).first()
          }).clip(bound)
    M_DSWDF_9 = M_DSWDF_9.where(M_DSWDF_9.gt(1), 1)
    ######################## Nutrient impact factor ########################
    m_outputFN = ee.Image(1).clip(bound)
    ######################## Material production and distribution ########################
    ################### Total Daily Assimilated Quantity SumDTGA ########################
    DTGA = ee.Image(0).clip(bound)
    P_J = p_j.filterMetadata('system:index', 'equals', str(i)).first()
    K_J = K_j.filterMetadata('system:index', 'equals', str(i)).first()
    par_i = PAR_i.filterMetadata('system:index', 'equals', str(i)).first()
    amax = AMAX.filterMetadata('system:index', 'equals', str(i)).first()
    dl = DL.filterMetadata('system:index', 'equals', str(i)).first()
    for j in range(3):
      SumTPS_j = ee.Image(0).clip(bound)

      for k in range(5):
        LGUSS_k = LAI.multiply(DIS5[k])
        Il_j_k = P_J.expression(
            'PDT > 0.2 ? (1 - p_j) * PAR_i * pow(e, -K_j * LGUSS_k) : 0',{
                'PDT': pdt,
                'p_j': P_J.select(j),
                'PAR_i': par_i.select(j),
                'e': math.e,
                'K_j': K_J.select(j),
                'LGUSS_k': LGUSS_k
            })
        Ila_j_k = Il_j_k.multiply(K_J.select(j))
        PS_j_k = Ila_j_k.expression(
            'PDT > 0.2 ? AMAX != 0 ? AMAX * (1 - exp(-0.5 * Ila_j_k / AMAX)) : 0 : 0',{
                'PDT': pdt,
                'AMAX': amax,
                'Ila_j_k': Ila_j_k
            })
        TPS_j_k = PS_j_k.multiply(WGUSS5[k])
        SumTPS_j = SumTPS_j.add(TPS_j_k)

      TPS_j = SumTPS_j.multiply(LAI)
      DTGA_j = TPS_j.multiply(WGUSS3[j])
      DTGA = DTGA.add(DTGA_j)
    DTGA = DTGA.multiply(dl).multiply(0.682)
    ######################################### Growth respiration rate RG ##########################################
    w1 = AWLVG.subtract(yAWLVG).abs()
    w2 = WST.subtract(yWST).abs()
    w3 = AROOTWT.subtract(yAROOTWT).abs()
    w4 = WSP.subtract(yWSP).abs()
    w = w1.add(w2).add(w3).add(w4)
    _w1 = w.expression(
        'w > 0 ? w1 / w : 0.25 ', {'w': w, 'w1': w1})
    _w2 = w.expression(
        'w > 0 ? w2 / w : 0.25 ', {'w': w, 'w2': w2})
    _w3 = w.expression(
        'w > 0 ? w3 / w : 0.25 ', {'w': w, 'w3': w3})
    _w4 = w.expression(
        'w > 0 ? w4 / w : 0.25 ', {'w': w, 'w4': w4})
    RG = _w1.multiply(0.72).add(_w2.multiply(0.69)).add(_w3.multiply(0.72)).add(_w4.multiply(0.73))
    ########################################## Maintenance respiration at standard reference temperature T0 RMT0 ##########################################
    RMT0 = AWLVG.multiply(0.03).add(WST.multiply(0.015)).add(AROOTWT.multiply(0.015)).add(WSP.multiply(0.01))
    ######################################### Maintenance of respiratory exertion RM ##########################################
    t24h = T24H.filterMetadata('system:index', 'equals', str(i)).first()
    tempRM = t24h.expression(
        'RMT0 * pow(2, (t24h - 25) / 10)',{
            'RMT0': RMT0,
            't24h': t24h
        })
    RM = tempRM.min(DTGA)
    W = (DTGA.multiply(m_outputSWDF1).subtract(RM)).multiply(RG).multiply(FCO2).divide(1 - 0.05)

    ABIOMASS = ABIOMASS.add(W)
    ################################################################################    Material distribution    #################################################################################
    pis = PIS.filterMetadata('system:index', 'equals', str(i)).first()
    pilvg = PILVG.filterMetadata('system:index', 'equals', str(i)).first()
    pisp = PISP.filterMetadata('system:index', 'equals', str(i)).first()

    yAWLVG = AWLVG
    yWST = WST
    yAROOTWT = AROOTWT
    yWSP = WSP
    yYIELD = YIELD
    TOPWT = ABIOMASS.multiply(pis)
    ATOPWT = ATOPWT.add((TOPWT.subtract(ATOPWT)).multiply(m_outputSWDF2))
    WLVG = ATOPWT.multiply(pilvg)
    WSP = ATOPWT.multiply(pisp)
    AROOTWT = ABIOMASS.subtract(ATOPWT)
    AROOTWT = AROOTWT.expression(
        'PDT >= 31 ? AROOTWT * 0.995 : AROOTWT',{
            'PDT': pdt,
            'AROOTWT': AROOTWT
        })
    AWLVG = AWLVG.add((WLVG.subtract(AWLVG)).multiply(m_outputFN))
    WST = ATOPWT.subtract(AWLVG).subtract(WSP)
    LAI = AWLVG.multiply(sla)
    rdrsh = LAI.expression(
        ' LAI <= 4 ? 0 : 0.03 * (LAI - 4) / 4',{
            'LAI': LAI
        })
    dLAIs = rdrsh.multiply(LAI)
    LAI = LAI.expression(
        '(dLAIs > 0) && (LAI > 0) ? LAI - dLAIs : LAI',{
            'dLAIs': dLAIs,
            'LAI': LAI
        })
    AWLVG = LAI.expression(
        '(dLAIs > 0) && (LAI > 0) ?  LAI / SLA : AWLVG',{
            'dLAIs': dLAIs,
            'LAI': LAI,
            'SLA': sla,
            'AWLVG': AWLVG
        })
    ######################## Corrected above-ground dry matter weight ########################
    ATOPWT = AWLVG.add(WST).add(WSP)
    ######################## Corrected dry matter weight ########################
    ABIOMASS = ATOPWT.add(AROOTWT)
    ######################## Status of supply of assimilates ########################
    YIELD = WSP.multiply(0.8).multiply(1.125)
    YIELD = YIELD.expression(
        'PDT >= 56 || YIELD <= yYIELD ? yYIELD : YIELD',{
            'PDT': pdt,
            'yYIELD': yYIELD,
            'YIELD': YIELD
        })
    YIELD = YIELD.expression(
        'PDT < 56 && DEM >= 550 ? YIELD * 0.7 : YIELD',{
            'PDT': pdt,
            'DEM': dem,
            'YIELD': YIELD
        }
    )
    ######################## RootModel ########################
    dGRT = AROOTWT.subtract(yAROOTWT)
    dGRT = dGRT.where(dGRT.lt(0), 0)
    ESW = dul.select(0).subtract(ll.select(0))
    m_outputSWDF2 = ESW.expression(
        'ESW < 0.25 ? sw > ll ? 4 * (sw - ll) / ESW : 0 : m_outputSWDF2',{
            'ESW': ESW,
            'sw': sw_0.filterMetadata('system:index', 'equals', str(i)).first(),
            'll': ll.select(0),
            'm_outputSWDF2': m_outputSWDF2
        })
    m_outputSWDF2 = m_outputSWDF2.where(m_outputSWDF2.gt(1), 1)
    SWDF1 = M_DSWDF_9.multiply(2)
    gdd = GDD.filterMetadata('system:index', 'equals', str(i)).first()
    RTDEP = RTDEP.expression(
          '0.2 < PDT && PDT < 31 ? RTDEP + GDD * 0.22 * MIN : RTDEP',{
              'PDT': pdt,
              'RTDEP': RTDEP,
              'GDD': gdd,
              'MIN': SWDF1.min(m_outputSWDF2)
          })
    RTDEP = RTDEP.where(RTDEP.gt(180), 180)
    cumdep = 0
    TRLDF = ee.Image(0).clip(bound)
    for l in range(10):
      cumdep += Layer
      sw_x = eval('sw_' + str(l)).filterMetadata('system:index', 'equals', str(i)).first()
      m_outputSWDF2 = ESW.expression(
          'ESW < 0.25 ? sw > ll ? 4 * (sw - ll) / ESW : 0 : 1',{
              'ESW': ESW,
              'sw': sw_x,
              'll': ll.select(l)
          })
      m_outputSWDF2 = m_outputSWDF2.where(m_outputSWDF2.gt(1), 1)
      RLDF1 = m_outputSWDF2.multiply(Layer).multiply(WR[l])


      RLDF = RLDF1
      RLDF = RLDF.expression(
          'cumdep <= RTDEP ? RLDF : (cumdep - RTDEP) < (Layer + 0.01) ? RLDF * (1 - (cumdep - RTDEP) / (Layer + 0.01)) : 0',{
              'cumdep': cumdep,
              'RTDEP': RTDEP,
              'RLDF': RLDF,
              'Layer': Layer
          })
      TRLDF = TRLDF.add(RLDF)
      if l == 0:
        rldf = RLDF
      else:
        rldf = rldf.addBands(RLDF)
    TRLDF = TRLDF.where(TRLDF.eq(0), 1)
    RLNEW = dGRT.multiply(1.05)


    for l in range(10):
      drlv = TRLDF.expression(
          'PDT > 0.2 ? 0.99 * dRLV + (RLDF / TRLDF) * (RLNEW / Layer) : 0',{
              'PDT': pdt,
              'dRLV': dRLV.select(l),
              'RLDF': rldf.select(l),
              'TRLDF': TRLDF,
              'RLNEW': RLNEW,
              'Layer': Layer
          })
      drlv = drlv.where(drlv.gt(10), 10)
      if l == 0:
        tempdRLV = drlv
      else:
        tempdRLV = tempdRLV.addBands(drlv)
    dRLV = tempdRLV

    Laimax = Laimax.max(LAI).rename('LAImax')

  YIELD = YIELD.rename('YIELD')
  ATOPWT = ATOPWT.rename('ATOPWT')
  AWLVG = AWLVG.rename('AWLVG')
  WST = WST.rename('WST')
  AROOTWT = AROOTWT.rename('AROOTWT')
  WSP = WSP.rename('WSP')
  yAWLVG = yAWLVG.rename('yAWLVG')
  yWST = yWST.rename('yWST')
  yAROOTWT = yAROOTWT.rename('yAROOTWT')
  yWSP = yWSP.rename('yWSP')
  ABIOMASS = ABIOMASS.rename('ABIOMASS')
  LAI = LAI.rename('LAI')
  RTDEP = RTDEP.rename('RTDEP')
  flag_chumiao = flag_chumiao.rename('flag_chumiao')
  WDurDays = WDurDays.rename('WDurDays')
  PDt_EXP = pdt


  return YIELD.addBands(ATOPWT).addBands(AWLVG).addBands(WST).addBands(AROOTWT).addBands(WSP).addBands(yAWLVG).addBands(yWST).\
      addBands(yAROOTWT).addBands(yWSP).addBands(ABIOMASS).addBands(LAI).addBands(RTDEP).addBands(flag_chumiao).addBands(dRLV).addBands(WDurDays).addBands(PDt_EXP).addBands(Laimax)






In [ ]:
asset_list = []

for i in range(60):
    asset_list.append('yourpathway' + str(i) + str(sowingYEAR))
    if ee.data.getInfo(asset_list[i]) is not None:
      ee.data.deleteAsset(asset_list[i])
      time.sleep(0.5)
      print(f"{asset_list[i]} has been deleted.")


ATOPWT = ee.Image(0).clip(bound)
AWLVG = ee.Image(0).clip(bound)
RTDEP = ee.Image(0).clip(bound)
flag_chumiao = ee.Image(1).clip(bound)
WST = ee.Image(0).clip(bound)
AROOTWT = ee.Image(0).clip(bound)
WSP = ee.Image(0).clip(bound)
LAI = ee.Image(0).clip(bound)
YIELD = ee.Image(0).clip(bound)
Laimax = ee.Image(0).clip(bound)
yAWLVG = AWLVG
yWST = WST
yAROOTWT = AROOTWT
yWSP = WSP

runnum = 20
N = 15

if N == 0:
  Num = 0
else:
  Num = (N+1) * runnum

while Num <= num:

  if Num >= runnum :
    YIELD = ee.Image(asset_list[N]).select(0)
    ATOPWT = ee.Image(asset_list[N]).select(1)
    AWLVG = ee.Image(asset_list[N]).select(2)
    WST = ee.Image(asset_list[N]).select(3)
    AROOTWT = ee.Image(asset_list[N]).select(4)
    WSP = ee.Image(asset_list[N]).select(5)
    yAWLVG = ee.Image(asset_list[N]).select(6)
    yWST = ee.Image(asset_list[N]).select(7)
    yAROOTWT = ee.Image(asset_list[N]).select(8)
    yWSP = ee.Image(asset_list[N]).select(9)
    ABIOMASS = ee.Image(asset_list[N]).select(10)
    LAI = ee.Image(asset_list[N]).select(11)
    RTDEP = ee.Image(asset_list[N]).select(12)
    flag_chumiao = ee.Image(asset_list[N]).select(13)
    dRLV = ee.Image(asset_list[N]).select(14)
    for k in range(15, 24):
      dRLV = dRLV.addBands(ee.Image(asset_list[N]).select(k))
    wDurDays = ee.Image(asset_list[N]).select(24)
    Laimax = ee.Image(asset_list[N]).select(26)
    N += 1



  WG = WheatGrowModel(ABIOMASS, dRLV, Q, Kt, dTmean, LwpCr, sw_0, sw_1, sw_2, sw_3, sw_4, sw_5, sw_6, sw_7, sw_8, sw_9, ll, dul, sat, p_j, \
                   PAR_i, K_j, AMAX, DL, T24H, PIS, PILVG, WR, PISP, SLA, GDD, Layer, PDT, Num, YIELD, ATOPWT, AWLVG, WST, \
                   AROOTWT, WSP, yAWLVG, yWST, yAROOTWT, yWSP, LAI, RTDEP, flag_chumiao, dem, wDurDays, Laimax)



  export_params1 = {
  'image': WG,
  'description': 'WG_' + str(N)+ str(sowingYEAR),
  'assetId': asset_list[N],
  'region': bound,
  'scale': proj['scale'],
  'maxPixels': 1e13
  }
  task1 = ee.batch.Export.image.toAsset(**export_params1)
  task1.start()
  while task1.status()['state'] in ['READY', 'RUNNING']:
    time.sleep(10)


  Num += runnum


